In [1]:
import numpy
import scipy
import os
# load TVB library
from tvb.simulator.lab import *
from tvb.basic.readers import FileReader

C:\Users\dinar\miniconda3\envs\incf\lib\site-packages\tvb\datatypes\surfaces.py:64: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


In [9]:
import numpy as np

# set up the simulation

# select the SJHMR3D model as the local model and define its parameters
# for delta series choose 
oscilator = models.ReducedSetHindmarshRose(r=numpy.array([0.006]), a=numpy.array([1.0]), b=numpy.array([3.0]), 
                                           c=numpy.array([1.0]), d=numpy.array([5.0]), s=numpy.array([4.0]), 
                                           xo=numpy.array([-1.6]), K11=numpy.array([0.5]), K12=numpy.array([0.1]), 
                                           K21=numpy.array([0.15]), sigma=numpy.array([0.3]), mu=numpy.array([2.2]), 
                                           variables_of_interest=["xi","alpha"])
oscilator.configure()

# for alpha series uncomment the below code
# oscilator = models.ReducedSetHindmarshRose(r=numpy.array([0.006]), a=numpy.array([1.0]), b=numpy.array([3.0]), 
#                                            c=numpy.array([1.0]), d=numpy.array([5.0]), s=numpy.array([4.0]), 
#                                            xo=numpy.array([-1.6]), K11=numpy.array([4.0]), K12=numpy.array([1.6]), 
#                                            K21=numpy.array([0.15]), sigma=numpy.array([0.4]), mu=numpy.array([2.2]), 
#                                            variables_of_interest=["xi","alpha"])
# oscilator.configure()

# set up the structural connectivity
mypath="C:/Users/dinar/Desktop/50healthy/DG_20120903"  # insert the path to where all subjects connectivities are stored
# subject="" # specify which subject to load

# load connectivity weights
reader = FileReader(file_path= mypath + '/weights.txt')
w= reader.read_array(dtype=numpy.float64, skip_rows=0, use_cols=None, matlab_data_name=None)

# load region centers
reader = FileReader(file_path=mypath + '/centres.txt')
rl = reader.read_array(dtype=str, skip_rows=0, use_cols=0, matlab_data_name=None)
c = reader.read_array(dtype=np.float64, skip_rows=0, use_cols=(1,), matlab_data_name=None)

# load connectivity tract lengths
reader = FileReader(file_path=mypath + '/distances.txt')
tl= reader.read_array(dtype=numpy.float64, skip_rows=0, use_cols=None, matlab_data_name=None)

# confige the connectivity
white_matter = connectivity.Connectivity(region_labels=rl, weights=w, tract_lengths=tl, centres=c)  

In [10]:
# specify the coupling function
# here a linear scaling function was used
# csf --> global coupling scaling factor
# is one of the parameters explored in this study
# for delta series it was varied from 0.05 to 0.25 in steps of 0.01
# for alpha series it was varied from 0.025 to 0.04 in steps of 0.001
# in this simulation one one value is chosen, but loop across all values to reproduce the findings of the paper
csf = 0.05
white_matter_coupling = coupling.Linear(a=np.array([csf]), b=np.array([0.0]))
white_matter_coupling.configure()

# specify a conduction speed to calculate time delays in the network 
# the 2nd parameter that was explored in this study
# for delta series it was varied from 20 to 100 in steps of 100
# for alpha series it was varied from 10 to 100 in steps of 10
# in this simulation one one value is chosen, but loop across all values to reproduce the findings of the paper


# specify a conduction speed to calculate time delays in the network 
# the 2nd parameter that was explored in this study
# for delta series it was varied from 20 to 100 in steps of 100
# for alpha series it was varied from 10 to 100 in steps of 10
# in this simulation one one value is chosen, but loop across all values to reproduce the findings of the paper
speed=100

# set up the integration scheme to solve the differential equations
# for delta series choose
heunint = integrators.HeunStochastic(dt=0.01220703125, noise=noise.Additive(nsig=np.array([1.0])))

# for alpha series uncomment the below code
# heunint = integrators.HeunStochastic(dt=0.05, noise=noise.Additive(nsig=numpy.array([0.001])))

# specify what data to record from the simulation using tvb monitory
# choose subsampling of the neural signal
p=3.90625 #<-- 256Hz
momo = monitors.SubSample(period=p)

# choose to generate BOLD signal using a hemodynamic response function
hrffunction=equations.MixtureOfGammas()
pb=500
mama = monitors.Bold(period=pb, hrf_kernel=hrffunction)

# put both monitors together
what_to_watch = (momo, mama)

# configure the simulation
sim = simulator.Simulator(model=oscilator, connectivity=white_matter,
                          coupling=white_matter_coupling, conduction_speed=speed,
                          integrator=heunint, monitors=what_to_watch)

sim.configure()

sim

# specify simulation length
# for delta series choose
# sim_length = 180000

,value
Type,Simulator
conduction_speed,100.0
connectivity,Connectivity gid: 95d7c811-44ae-48ad-a4bc-22e453b5f20b
coupling,Linear gid: 5b1ab672-e3d4-4d96-9394-e8d892616c42
gid,UUID('86a8e33a-ea97-44db-9bfa-35f3908d340d')
initial_conditions,None
integrator,HeunStochastic gid: 6a41a275-13a9-4d33-9df3-c6f3133f4d8f
model,ReducedSetHindmarshRose gid: 55acf9b0-53bf-410b-b430-e39fdb15d381
monitors,"(, )"
simulation_length,1000.0


In [16]:
# specify simulation length
# for delta series choose
sim_length = 1000

# perform the simulation
subs_data = []
subs_time = []
bold_data = []
bold_time = []

for subs, bold in sim(simulation_length=sim_length):
    if subs is not None:
        subs_time.append(subs[0])
        subs_data.append(subs[1])
        
        if subs_data:
            np.save('subs_data.txt', numpy.sum(numpy.array(subs_data),axis=3))
        
        if subs_time:
            np.save('subs_time.txt', np.array(subs_time))
    
    if bold is not None:
        bold_time.append(bold[0])
        bold_data.append(bold[1])
        
        if bold_data:
            np.save('bold_data.txt', np.array(bold_data))

        if bold_time:
            np.save('bold_time.txt', np.array(bold_time))


# save the simulated time series
# declare a path where data should be saved
save_path=""
SUBS  = numpy.sum(numpy.array(subs_data),axis=3)
TSUBS = numpy.array(subs_time)
BOLD  = numpy.array(bold_data)
TBOLD = numpy.array(bold_time)
subsfile = 'csf_'+str(csf) +'_speed_' +str(speed)
scipy.io.savemat(subsfile, mdict={'subs_data': SUBS, 'subs_time': TSUBS ,'bold_data': BOLD, 'bold_time': TBOLD })

In [18]:
fc = np.corrcoef(BOLD[:, 0, :, 0].T)
fc

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])